In [1]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
data_dir = r'C:\Users\haris\AI\ML\Projects\Data\Cars Dataset'

img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator( horizontal_flip = True,
                              validation_split=0.244,
                              rotation_range=20, 
                              width_shift_range=0.1, 
                              height_shift_range=0.1,
                              fill_mode='nearest')

# datagen2 = ImageDataGenerator(rescale=1./255, horizontal_flip = False, validation_split=0.242)

train_images = datagen.flow_from_directory(
                                          os.path.join(data_dir, 'train'),
                                          target_size = img_size,
                                          class_mode = "categorical",
                                          shuffle = True,
                                          batch_size = batch_size,
                                          subset = 'training')

valid_images = datagen.flow_from_directory(os.path.join(data_dir, 'train'),
                                           target_size = img_size,
                                           class_mode = "categorical",
                                           shuffle = False,
                                           batch_size = batch_size,
                                           subset = 'validation')
test_images = datagen.flow_from_directory(os.path.join(data_dir, 'test'),
                                           target_size = img_size,
                                           class_mode = "categorical",
                                           shuffle = False,
                                           batch_size = batch_size)

Found 2537 images belonging to 7 classes.
Found 815 images belonging to 7 classes.
Found 813 images belonging to 7 classes.


In [3]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor="val_accuracy",patience  = 5,verbose=1, restore_best_weights=True)

history = model.fit(
    train_images,
    validation_data=valid_images,
    batch_size=batch_size,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
80/80 [==============================] - 32s 334ms/step - loss: 1.7646 - accuracy: 0.3591 - val_loss: 1.4764 - val_accuracy: 0.4638
Epoch 2/20
80/80 [==============================] - 26s 319ms/step - loss: 1.4544 - accuracy: 0.4777 - val_loss: 1.3940 - val_accuracy: 0.4933
Epoch 3/20
80/80 [==============================] - 26s 319ms/step - loss: 1.3990 - accuracy: 0.4923 - val_loss: 1.3995 - val_accuracy: 0.4945
Epoch 4/20
80/80 [==============================] - 26s 320ms/step - loss: 1.3062 - accuracy: 0.5329 - val_loss: 1.3719 - val_accuracy: 0.5362
Epoch 5/20
80/80 [==============================] - 26s 320ms/step - loss: 1.2652 - accuracy: 0.5388 - val_loss: 1.2736 - val_accuracy: 0.5571
Epoch 6/20
80/80 [==============================] - 26s 319ms/step - loss: 1.1991 - accuracy: 0.5637 - val_loss: 1.3932 - val_accuracy: 0.5055
Epoch 7/20
80/80 [==============================] - 26s 320ms/step - loss: 1.1778 - accuracy: 0.5751 - val_loss: 1.3675 - val_accuracy: 0.5399

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')  
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# # Load the test data
# test_images, test_labels, num_classes = load_and_preprocess_data('test')

# # Evaluate the model on the test data
# test_loss, test_accuracy = model.evaluate(test_images, test_labels)
# print(f"Test Loss: {test_loss}")
# print(f"Test Accuracy: {test_accuracy}")

# # Make predictions on the test data
# test_predictions = model.predict(test_images)
# test_predictions_classes = np.argmax(test_predictions, axis=1)
# true_classes = np.argmax(test_labels, axis=1)

# # Calculate the confusion matrix
# arr = confusion_matrix(true_classes, test_predictions_classes)

# # Print the classification report
# class_names = [str(i) for i in range(num_classes)]  # Use numeric class labels for the test dataset
# classification_rep = classification_report(true_classes, test_predictions_classes, target_names=class_names)
# print("Classification Report:\n", classification_rep)


In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(arr, range(7), range(7))
sn.heatmap(df_cm, annot=True,fmt="d", cmap="YlGnBu") 

In [ ]:
# # Create a DataFrame from the classification report
# classification_dict = classification_report(true_classes, test_predictions_classes, target_names=class_names, output_dict=True)
# classification_df = pd.DataFrame(classification_dict).transpose()

# # Save the DataFrame as an Excel file
# excel_path = os.path.join(r'C:\Users\haris\AI\ML\Projects\Data\Cars Dataset', 'classification_report.xlsx')
# classification_df.to_excel(excel_path, index=True)

# print(f"Classification report saved as Excel to {excel_path}")

In [ ]:
model.summary()